# 验证码生成器

In [1]:
from captcha.image import ImageCaptcha  # pip install captcha
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
# 验证码中的字符, 就不用汉字了
number = ['0','1','2','3','4','5','6','7','8','9']
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
ALPHABET = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
# 验证码一般都无视大小写；验证码长度4个字符
def random_captcha_text(char_set=number+alphabet+ALPHABET, captcha_size=4):
	captcha_text = []
	for i in range(captcha_size):
		c = random.choice(char_set)
		captcha_text.append(c)
	return captcha_text

# 生成字符对应的验证码
def gen_captcha_text_and_image():
	image = ImageCaptcha()
	captcha_text = random_captcha_text()
	captcha_text = ''.join(captcha_text)
	captcha = image.generate(captcha_text)
	#image.write(captcha_text, captcha_text + '.jpg')  # 写到文件
	captcha_image = Image.open(captcha)
	captcha_image = np.array(captcha_image)
	return captcha_text, captcha_image

if __name__ == '__main__':
    text, image = gen_captcha_text_and_image()
    f = plt.figure()
    ax = f.add_subplot(111)
    ax.text(0.1, 0.9,text, ha='center', va='center', transform=ax.transAxes)
    plt.imshow(image)
    plt.show()

# 图片hash

In [3]:
from PIL import Image
import sys

# 该算法对图片太敏感, 比如图片旋转等处理过, hash值就大不同了
def get_hash(image_path):
    im = Image.open(image_path)
    #antialias 抗锯齿
    #convert 转换 L 代表灰度
    im = im.resize((8, 8), Image.ANTIALIAS).convert('L')
    #avg:像素的平均值
    avg=sum(list(im.getdata()))/64.0
 
    #avg和每个像素比较，得到长度64的字符串
    str=''.join(map(lambda i: '0' if i<avg else '1', im.getdata()))
    #str切割，每4个字符一组，转成16进制字符
    return ''.join(map(lambda x:'%x' % int(str[x:x+4],2), range(0,64,4)))

# phash, 余弦hash, 可以识别25%以内的变形
def get_phash(image_path):
    import imagehash
    phash = imagehash.phash(Image.open(image_path))
    return phash

def get_hamming_distance(x, y):
    return bin(x^y).count('1')

# 需要编译安装pHash包, 依赖CImg
class pHash(object):
    def __init__(self):
        self._lib = ctypes.CDLL('/opt/local/lib/libpHash.dylib', use_errno=True)

    def dct_imagehash(self, path):
        phash = ctypes.c_uint64()
        if self._lib.ph_dct_imagehash(path, ctypes.pointer(phash)):
            errno_ = ctypes.get_errno()
            err, err_msg = (errno.errorcode[errno_], os.strerror(errno_))  if errno_ else ('none', 'errno was set to 0')
            print(('Failed to get image hash ({!r}): [{}] {}').format(path, err, err_msg), file=sys.stderr)
            return None
        return phash.value
    
    def hamming_distance(self, hash1, hash2):
        return self._lib.ph_hamming_distance(*map(ctypes.c_uint64, [hash1, hash2]))
    
split_count = 8  # 每个64位的phash值分为八段，每段8位

def split(key, split_count):
    pre_length = 64 / split_count
    return [key[i * pre_length: (i + 1) * pre_length] for i in range(split_count)]

class ImageManager(object):
    def __init__(self):
        self.phash = pHash() # 就是上面那个pHash类
        self.phash_cache = [defaultdict(list) for i in range(split_count)] #
        self.init_phash_map()

    def init_phash_map(self):
        #我是把所有的phash存在sqlite里边，这边取出所有的Image
        for image in Image.select():
            self.add_to_image_cache(image)

    def add_to_image_cache(self, image):
        # 将hash值分割为8段
        key_split = split(bin(int(image.phash))[2:].rjust(64, '0'), split_count)
        for index, k in enumerate(key_split):
            self.phash_cache[index][k].append(image)

    def has_same(self, ori_image):
        phash = ori_image.phash
        key_split = split(bin(int(phash))[2:].rjust(64, '0'), split_count)
        result = set()
        for index, k in enumerate(key_split):
            if k in self.phash_cache[index]:
                for image in self.phash_cache[index][k]:
                    distance = self.distance(int(phash), int(image.phash))
                    if distance < 5 and ori_image.key != image.key:
                        result.add(image)
        if result:
            return True,list(result)
        return False,[]

if __name__ == '__main__':
    image = '/Users/work/Downloads/ooxx/006HcaCrgy1fld5rzv14aj30jg0jg111.jpg'
    hash = get_hash(image)
    print(hash)
    phash = get_phash(image)
    print(phash)
    ham = get_hamming_distance(int(hash, 16), 0x7c7c080004fffee5)
    print(ham)

7c7c080004fffee6
eb52a355e4729568
2


# simhash 文本重复率查询

In [8]:
import numpy as np
import jieba
import hashlib
import re

class Repeat:
    REX_CH = re.compile(u'[\u4e00-\u9fa5]+')    # 中文
    REX_EN = re.compile('[A-Za-z]+')        # 英文

    cut_func = jieba.cut
    @classmethod
    def hash2bin(cls, hash):
        d = ''
        for i in hash:
            try:
                if int(i) > 7:
                    d = d + '1'
                else:
                    d = d + '0'
            except ValueError:
                d = d + '1'
        return d

    @classmethod
    def hash_bin(cls, s):
        h = hashlib.md5(s.encode()).hexdigest()
        return cls.hash2bin(h)

    @classmethod
    def hist(cls, cut):
        _cut = {x: 0 for x in set(cut)}
        for i in cut:
            _cut[i] += 1
        return {cls.hash_bin(k): v/len(cut) for k, v in _cut.items()}

    @classmethod
    def simhash(cls, s, RE=None, cut_func=None):
        if RE:
            REX = RE
        else:
            REX = re.compile(u'[\u4e00-\u9fa5]+')
        if not cut_func:
            cut_func = cls.cut_func

        cut = [x for x in cut_func(s) if re.match(REX, x)]

        ver = [[v * (int(x) if int(x) > 0 else -1) for x in k] for k, v in cls.hist(cut).items()]
        ver = np.array(ver)
        ver_sum = ver.sum(axis=0)
        sim = ''.join(['1' if x > 0 else '0' for x in ver_sum])
        return sim

    @staticmethod
    def _hamming(s1, s2):
        d = [1 if a1 == a2 else 0 for a1, a2 in zip(s1, s2)].count(1)
        return d


xxx = '''
1993年，南京大学有这样一个男生寝室，四个男生都没有女朋友，
于是搞了个组合叫“名草无主四大天王”。这四大天王坚持每晚举行“卧谈会”，
从各种学术上讨论如何摆脱光棍状态。这一年的11月，校园的梧桐树落叶凋零，令他们分外伤情。
他们在11日这一天晚上卧谈时，符号学的灵感突然登门造访。
11月11日，四个1字排开，不正是好像四根光秃秃的棍子吗？这四根光棍不正是在巧妙地诉说着“名草无名四大天王”的凄凉吗？
'''
yyy = '''
知乎上有个提问，小时候缺爱的女孩子，长大后该怎么办？
或许在我这里，只是希望一直有人陪。
喜宝说，我想要很多很多的爱，要不就是很多很多的钱，实在不行，有健康也是好的。
我有个坏毛病，经常会半夜饿到不行，爬起来找吃的。是真的饿到胃疼，有时候直接饿醒了，每次看到电影里的台词，睡着了就不饿了，我是压根不相信。
为什么会半夜饿？
究其原因，是大学的时候没人陪我吃饭，每次都是一直等到有人陪我的时候，我才会去吃饭，最后把自己饿到胃疼，久而久之，就渐渐习惯了熬到很晚才吃饭。
我不喜欢一个人吃饭，也不喜欢一个人逛街，更不喜欢一个人呆着，可是成长啊，往往是越不喜欢的便越要学会接受它。
（二）
讲讲上一段恋爱吧。
我和他认识的时候，是因为贴吧聚餐，他主动找我要的微信，附带一个如沐春风般的笑容。
我一直以为他是被我的美色打动，后来问他原因。
他说，他第一次看见那么能吃的女孩子，他惊呆了，可是有觉得看我吃饭很意思，仿佛食物都有了灵魂，让人的心情莫名的好了起来。
我们初相识，是因为他看见了我饿死鬼投胎的吃相。
我们在一起，是因为他厨艺很好，好到什么程度呢？就是那种你吃过一顿，就能惦记一辈子的感觉。即便是现在回忆起他来，我的味蕾都会有反应。
他总是给我做很多很多好吃的，午后阳光从窗子洒进来，窗帘是淡绿色的小碎花，空气里弥漫着饭香味，我们两个人坐在桌前，一边吃饭，一边聊天。
我喜欢和他一起手挽着手去菜市场买菜，西红柿土豆黄瓜小白菜，手里拎着的这些果蔬食物，就好像我拥有的全世界。
有一次，我们从菜市场回去的路上，明明是艳阳高照的天气，却突然间下起了冰雹，那是他第一次看见冰雹，被砸了一下之后，便立马丢了手里的菜，双手护住我，我傻了吧唧的去捡菜，被砸了一身。
他立马臭骂了我一顿，说我是他见过，最好吃的女孩子了。
'''

zzz = '''
知乎上有个提问，小时候缺爱的女孩子，长大后该怎么办？
或许在我这里，只是希望一直有人陪。
喜宝说，我想要很多很多的爱，要不就是很多很多的钱，实在不行，
我有个坏毛病，经常会半夜饿到不行，爬起来找吃的。是真的饿到胃疼，有时候直接饿醒了，每次看到电影里的台词，睡着了就不饿了，我是压根不相信。
究其原因，是大学的时候没人陪我吃饭，每次都是一直等到有人陪我的时候，我才会去吃饭，最后把自己饿到胃疼，久而久之
我不喜欢一个人吃饭，也不喜欢一个人逛街，更不喜欢一个人呆着，可是成长啊，往往是越不喜欢的便越要学会接受它。
我和他认识的时候，是因为贴吧聚餐，他主动找我要的微信，附带一个如沐春风般的笑容。
我一直以为他是被我的美色打动，后来问他原因。
他说，他第一次看见那么能吃的女孩子，他惊呆了，可是有觉得看我吃饭很意思，仿佛食物都有了灵魂，让人的心情莫名的好了起来。
我们初相识，是因为他看见了我饿死鬼投胎的吃相。
我们在一起，是因为他厨艺很好，好到什么程度呢？就是那种你吃过一顿，就能惦记一辈子的感觉。即便是现在回忆起他来，我的味蕾都会有反应。
他总是给我做很多很多好吃的，午后阳光从窗子洒进来，窗帘是淡绿色的小碎花，空气里弥漫着饭香味，我们两个人坐在桌前，一边吃饭，一边聊天。
我喜欢和他一起手挽着手去菜市场买菜，西红柿土豆黄瓜小白菜，手里拎着的这些果蔬食物，
有一次，我们从菜市场回去的路上，明明是艳阳高照的天气，却突然间下起了冰雹，那是他第一次看见冰雹，被砸了一下之后，便立马丢了手里的菜，双手护住我，我傻了吧唧的去捡菜，被砸了一身。
他立马臭骂了我一顿，说我是他见过，最好吃的女孩子了。
'''
ccc = zzz * 10

h1 = Repeat.simhash(yyy)
h2 = Repeat.simhash(zzz)
h3 = Repeat.simhash(ccc)
print('h1:{0}\nh2:{1}\nh3:{2}'.format(h1, h2, h3))
print(Repeat._hamming(h3, h2))

h1:00101101001010110001100000101110
h2:00100101001010110000100000101110
h3:00100101001010110000100000101110
32


# bk树算法

In [3]:
import editdistance


class Node(object):
    def __init__(self, index):
        self.index = index
        self.value = self.get_value(index)
        self.children = {}

    @staticmethod
    def get_value(index):
        return str(index)

    def __str__(self):
        return self.value


class Tree(object):
    def __init__(self, values=None, dis_func=None):
        self.root = None
        self.dis_func = dis_func
        if values:
            for value in values:
                self.add(value)

    def add(self, value):
        if self.root is None:
            self.root = Node(value)
        else:
            node = Node(value)
            top = self.root
            distance = self._distance(node, top)
            while distance in top.children.keys():
                top = top.children[distance]
                distance = self._distance(node, top)
            if distance == 0:
                print('distance is 0, so they are same')
            else:
                top.children[distance] = node
                node.parent = top

    def search(self, value, max_distance=2):
        candidates = [self.root]
        found = []

        while len(candidates) > 0:
            node = candidates.pop(0)
            distance = self._distance(node.value, value)

            if distance > max_distance:
                pass
            else:
                found.append(node.value)
            candidates.extend(node.children[dis] for dis in node.children.keys()
                              if abs(distance - max_distance) <= dis <= distance + max_distance)
        return found

    def _distance(self, a, b):
        a = a.value if isinstance(a, Node) else a
        b = b.value if isinstance(b, Node) else b
        if self.dis_func:
            return self.dis_func(a, b)
        else:
            return editdistance.eval(a, b)

    def __str__(self):
        candidates = [self.root]
        found = []
        while len(candidates) > 0:
            node = candidates.pop(0)
            found.append(node.value)
            candidates.extend(nd for nd in node.children.values())
        return str(found)